In [1]:
import torch
import json
from embed_llm.models.augmented_model import EmbedAugPipeline
from embed_llm.generation.evaluation import  ensure_reproducibility
from bertviz import head_view, model_view # type: ignore
from embed_llm.models.mistral.generate import get_attention

/home/hippolytepilchen/micromamba/envs/llm_embed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/hippolytepilchen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
ensure_reproducibility(29)

In [3]:

#Must have a params json for pipeline
llm_path = '/lustre/scwpod02/client/kyutai-interns/hippop/models/mistral_7B'

run_name = 'pretrain_llm_trained_rec_singpassage_054f63f8'
last_ckpt = '040000' # '008500' #'010000' 

run_name = '128_SL_FN_False_1_MLP_True_CA_16_CAL_False_SKV_True_DB'
last_ckpt = '010000'


device = 'cuda' if torch.cuda.is_available() else 'cpu'

if torch.cuda.device_count() > 1:
    device = torch.device('cuda:0')
    print(f'Using {device} for loading')

w_embeds = True
max_batch_size = 4

Using cuda:0 for loading


In [4]:
pipeline: EmbedAugPipeline = EmbedAugPipeline.load_inference_model(
    llm_path=llm_path,
    ckpt_path="/lustre/scwpod02/client/kyutai-interns/hippop/tmp/"
    + run_name
    + "/checkpoints/checkpoint_"
    + last_ckpt,
    device=device,
    llm_name="Mistral7B",
    embed_model_name="NVEmbed",  # Not used if pretrainde ckpt available
    max_batch_size=max_batch_size,
)
print('Pipeline w prefix or not ? ', pipeline.pipeline_args.w_prefix_prompt)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.02s/it]


Loading cross att state dict
Not only LoRA weights found in the checkpoint. Skipping other weights.
Loading LoRA weights...
If n_layers is 1, hidden_dim must be equal to out_dim, 
 but hidden_dim is not equal to out_dim so hidden_dim is set to out_dim
Loading MLP projector
Pipeline w prefix or not ?  False


In [5]:
# Flipping attempts
w_embeds = True
temp = 0
max_tokens = 128
i_token_to_flip = -1
if w_embeds:
    pipeline.pipeline_args.w_embeds = True
else:
    pipeline.pipeline_args.w_embeds = False

temp = [temp] * max_tokens   
if max_tokens > i_token_to_flip >= 0:
    temp[i_token_to_flip] = 1000
    
prompt = ''
text_conditioning ='Mario Bortolazzi (born 10 January 1965, in Verona) is an Italian professional football coach and a former player, who played as a midfielder. \
    \n\nHe played 12 seasons (241 games, 14 goals) in the Serie A for ACF Fiorentina, A.C. Milan, Hellas Verona F.C., Atalanta B.C. and Genoa C.F.C.'
        # \n\nIn his coaching career he has so far has always been an assistant to his former Milan teammate Roberto Donadoni.\
        #     \n\nHonours\n\n - Milan\n - Serie A champion: 1987–88.\n\n - Genoa\n - Anglo-Italian Cup winner: 1995–96.'



            
generated_sequence, attn_or_logprobs, embeddings = pipeline.generate(prompt_pre_embed = ['In other words, background: ',''], 
                                    prompt_post_embed = [' is just another way of saying: ',''],
                                    text_conditioning = [text_conditioning]*2, 
                                    temperature = temp, 
                                    max_tokens = max_tokens,
                                    truncate_double_space = False,
                                    random_flip = i_token_to_flip,
                                    device = device,
                                    device_generation = device if torch.cuda.device_count() <= 1 else torch.device('cuda:1'),
                                    return_embeddings = True)
print(generated_sequence)

/home/hippolytepilchen/.cache/huggingface/modules/transformers_modules/nvidia/NV-Embed-v2/5130cf1daf847c1bacee854a6ef1ca939e747fb2/modeling_nvembed.py:349: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),


Prompt Pre Embed: [8, 0]
Prompt Post Embed: [10, 1]
['\n\nMario Alberto Bortolazzi (born 12 January 1964) is a former professional footballer who played as a midfielder. He played for 12 seasons at A.C. Milan, winning the Serie A championship in 1991–92, and the Coppa Italia in 1993–94. He was transferred to Genoa C.F.C. for 500,000 lire, and made his debut on 25 September 1995, in a match against Fiorentina at', 'Mario Bortolazzi\n\nMario Bortolazzi (born 12 January 1965 in Genoa) is an Italian former professional football player and coach. He played 12 seasons in Serie A, for A.C. Milan, Atalanta, Fiorentina, and Hellas Verona. He played 246 games in Serie A, scoring 14 goals. 1990–91 season, he played 12 games in Serie B, for A.C. Milan. 1991–92 season,']


In [6]:
n_passages = 20

lim_toks = 128
eval_data = '/lustre/scwpod02/client/kyutai-interns/datasets/modular_finetuning/enwiki-20220120_valid.jsonl'
train_data = '/lustre/scwpod02/client/kyutai-interns/datasets/modular_finetuning/enwiki-20220120_train.jsonl'
train_passage = []
valid_passage = []

with open(train_data, 'r') as f:
    for i, line in enumerate(f):
        if i == n_passages:
            break
        train_passage.append(pipeline.tokenizer.decode(pipeline.tokenizer.encode(json.loads(line)['text'].split('\n\n')[1], eos = True, bos = True)[:lim_toks]))
  
with open(eval_data, 'r') as f:
    for i, line in enumerate(f):
        if i == n_passages:
            break
        valid_passage.append(pipeline.tokenizer.decode(pipeline.tokenizer.encode(json.loads(line)['text'].split('\n\n')[1], eos = True, bos = True)[:lim_toks]))
        

In [41]:
w_embeds = True
temp = 0
max_tokens = 128
i_token_to_flip = -1
if w_embeds:
    pipeline.pipeline_args.w_embeds = True
else:
    pipeline.pipeline_args.w_embeds = False

temp = [temp] * max_tokens   
if max_tokens > i_token_to_flip >= 0:
    temp[i_token_to_flip] = 1000
# 1 information in the doc which enables to answer the question but not good response often in-context
# 2 information in the doc which enables to answer the question and good response often in-context
# 3 Hard negative passage
# 4 Same

prompt_prefix = "Query: "
prompts = ['who has most followers on instagram in world','who did the united states win its independence from', 'locations for the film an englishman who went up a hill', 'who is the valley of the dolls based on']
prompts = [prompt_prefix + prompt + '\nAnswer:' for prompt in prompts]

conditioning = ["This list contains the top 50 accounts with the most followers on the photo and video-sharing social platform Instagram. As of July 2019, the most followed user is Instagram's own account, with over 308 million followers. Cristiano Ronaldo is the most followed individual, with over 177 million followers. Fifteen accounts have exceeded 100 million followers on the site.",
                     "During the American Revolution, the legal separation of the thirteen colonies from Great Britain in 1776 actually occurred on July 2, when the Second Continental Congress voted to approve a resolution of independence that had been proposed in June by Richard Henry Lee of Virginia declaring the United States independent from Great Britain's rule. After voting for independence, Congress turned its attention to the Declaration of Independence, a statement explaining this decision, which had been prepared by a Committee of Five, with Thomas Jefferson as its principal author. Congress debated and revised the wording of the Declaration, finally approving it two days later on July 4. A day earlier, John Adams had written to his wife Abigail",
                     'The village was a primary location for the making of the film \"The Englishman Who Went Up a Hill But Came Down a Mountain\", which starred Hugh Grant. The hilltop scenes were filmed on the Gyrn, the long hill that overlooks the village. It was also featured in \"Monk\'s Hood\", an episode of \"The Cadfael Chronicles\"',
                     'Valley of the Dolls is the first novel by American writer Jacqueline Susann. Published in 1966, the book was the biggest selling novel of its year. To date, it has sold more than 31 million copies, making it one of the best-selling works in publishing history.']

# answers = ['Instagram','Great Britain',"Llansilin in Powys",["Judy Garland", "Carole Landis", "Dean Martin", "Ethel Merman"]]

# conditioning = 'Kyutai is a non-profit laboratory dedicated to open research in AI, founded in November 2023 by the iliad Group, CMA CGM and Schmidt Sciences.'
# prompts =  prompt_prefix +'Query: when was founded Kyutai?\nAnswer: '



if w_embeds:
    pipeline.pipeline_args.w_embeds = True
else:
    pipeline.pipeline_args.w_embeds = False
pipeline.model.llm.pos_to_keep = []
generated_sequence, logprobs = pipeline.generate(prompts = prompts, 
                                    text_conditioning = conditioning, 
                                    temperature = temp, 
                                    max_tokens = max_tokens,
                                    truncate_double_space = False,
                                    random_flip = i_token_to_flip,
                                    device = device,
                                    device_generation = device if torch.cuda.device_count() <= 1 else torch.device('cuda:1'),
                                    return_embeddings = False)
print(generated_sequence)


## Vizualisation

In [ ]:
self_att_weights, tokens = get_attention(generated_sequence[0], embeddings, pipeline.tokenizer, pipeline.model.llm, n_tokens = 20)
# self_att_weights, tokens = get_attention('He played 12 seasons (243 games, 14 goals) in the Serie A for A.C. Milan, A.F.C. Fiorentina, Hellas Verona, Atalanta B.C. and Genoa C.F.C.', embeddings, pipeline.tokenizer, pipeline.model.llm, n_tokens = 20)

head_view(self_att_weights, tokens)

In [ ]:
self_att_weights, tokens = get_attention(generated_sequence[0], embeddings, pipeline.tokenizer, pipeline.model.llm, n_tokens = 20)
model_view([att[:,:,1:,1:] for att in self_att_weights], [tokens[0]] + tokens[2:])

In [ ]:

mean_per_head_att = [torch.mean(att, dim = 1, keepdim = True) for att in self_att_weights]
overall_mean = torch.mean(torch.stack(self_att_weights), dim = 0)

head_view(mean_per_head_att, tokens)
# head_view([overall_mean], tokens)

## Evaluation metrics

In [ ]:
print(stop)
temperatures = [0, 0.5, 0.7, 1, 1.5]
max_tokens = 150

results_generation = {'0':{'train': {'word_prompt':{}, 'empty_prompt':{}}, 'valid': {'word_prompt':{}, 'empty_prompt':{}}}, 
                        '0.5':{'train': {'word_prompt':{}, 'empty_prompt':{}}, 'valid': {'word_prompt':{}, 'empty_prompt':{}}},
                        '0.7':{'train': {'word_prompt':{}, 'empty_prompt':{}}, 'valid': {'word_prompt':{}, 'empty_prompt':{}}},
                        '1':{'train': {'word_prompt':{}, 'empty_prompt':{}}, 'valid': {'word_prompt':{}, 'empty_prompt':{}}},
                        '1.5':{'train': {'word_prompt':{}, 'empty_prompt':{}}, 'valid': {'word_prompt':{}, 'empty_prompt':{}}}}


n_passages = len(train_passage)
assert n_passages == len(valid_passage)

for temp in temperatures:
    print(f'Temperature: {temp}')    
    generated_sequences = []
    
    for i in range(0, n_passages, max_batch_size):
        passage = train_passage[i:i+max_batch_size]
        generated_sequence, logprobs = pipeline.generate(prompts = [text.split(' ')[0] for text in passage], 
                                    text_conditioning = passage, 
                                    temperature = temp, 
                                    max_tokens = max_tokens,
                                    truncate_double_space = False,
                                     device = device,
                                    device_generation = device if torch.cuda.device_count() <= 1 else torch.device('cuda:1'))
           
        generated_sequences.extend(generated_sequence)
    results_generation[str(temp)]['train']['word_prompt'] = {'seq':generated_sequences}
    print('Train Passage:', passage)
    print('Train Generated:', generated_sequence)
    generated_sequences = []
    for i in range(0, n_passages, max_batch_size):
        passage = train_passage[i:i+max_batch_size]
        generated_sequence, logprobs = pipeline.generate(prompts = [''] * len(passage), 
                                    text_conditioning = passage, 
                                    temperature = temp, 
                                    max_tokens = max_tokens,
                                    truncate_double_space = False,
                                     device = device,
                                    device_generation = device if torch.cuda.device_count() <= 1 else torch.device('cuda:1'))
           
        generated_sequences.extend(generated_sequence)    
    results_generation[str(temp)]['train']['empty_prompt'] = {'seq':generated_sequences}
    

    generated_sequences = []
    for i in range(0, n_passages, max_batch_size):
        passage = valid_passage[i:i+max_batch_size]
        generated_sequence, logprobs = pipeline.generate(prompts = [text.split(' ')[0] for text in passage], 
                                    text_conditioning = passage, 
                                    temperature = temp, 
                                    max_tokens = max_tokens,
                                    truncate_double_space = False,
                                     device = device,
                                    device_generation = device if torch.cuda.device_count() <= 1 else torch.device('cuda:1'))
           
        generated_sequences.extend(generated_sequence)    
    results_generation[str(temp)]['valid']['word_prompt'] = {'seq':generated_sequences}
    
    generated_sequences = []
    for i in range(0, n_passages, max_batch_size):
        passage = valid_passage[i:i+max_batch_size]
        generated_sequence, logprobs = pipeline.generate(prompts = [''] * len(passage), 
                                    text_conditioning = passage, 
                                    temperature = temp, 
                                    max_tokens = max_tokens,
                                    truncate_double_space = False,
                                     device = device,
                                    device_generation = device if torch.cuda.device_count() <= 1 else torch.device('cuda:1'))
           
        generated_sequences.extend(generated_sequence)    
    results_generation[str(temp)]['valid']['empty_prompt'] = {'seq':generated_sequences}
    print('Valid Passage:', passage)
    print('Valid Generated:', generated_sequence)
        

In [ ]:
metrics = []
for temp in results_generation.keys():
    for split in results_generation[temp].keys():
        for prompt_type in results_generation[temp][split].keys():
            generated_sequences = results_generation[temp][split][prompt_type]['seq']
            if prompt_type == 'empty_prompt':
                gt_passage = train_passage if split == 'train' else valid_passage
                overlap = word_overlap(gt_passage, generated_sequences)
                bleu_score = get_bleu_score(gt_passage, generated_sequences)
            elif prompt_type == 'word_prompt':
                gt_passage = train_passage if split == 'train' else valid_passage
                gt_passage = [' '.join(text.split(' ')[1:]) for text in gt_passage]
                overlap = word_overlap(gt_passage, generated_sequences)
                bleu_score = get_bleu_score(gt_passage, generated_sequences)
   
            print(f'Temperature: {temp}, Split: {split}, Prompt Type: {prompt_type}, Overlap: {overlap}', 'Bleu Score:', bleu_score)
            metrics.append({'temp': temp, 'split': split, 'prompt_type': prompt_type, 'overlap': overlap, 'bleu_score': bleu_score})
            
# with open(f'{ckpt_path}/results_generation.json', 'w') as f:
#     json.dump(metrics, f)

## Old

In [ ]:
llm_path = '/lustre/scwpod02/client/kyutai-interns/hippop/models/mistral_7B'
#Must have a params json for pipeline

# No embeddings:
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/no_embed_bs16_lr5e-5Mistral7B88d0b42410aa4ec12025/checkpoints/checkpoint_002500'

# Length tokens:
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_512t_Mistral7B20ed0018b2a84fba09c4/checkpoints/checkpoint_005000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_256t_Mistral7Be9ffc00fa42bedbc50d0/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_128t_Mistral7B226729d875c65b331ef8/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_64t_Mistral7B9bbea1b3b8dc23079b04/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_32t_Mistral7Bccbc3f29d69bd124c6cf/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_16t_Mistral7B7bc7dcc2ba28873eda96/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/mean_not_causal/checkpoints/checkpoint_007500'


# # Continuation:
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/continuation_Mistral7B20ed0018b2a84fba09c4/checkpoints/checkpoint_006000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/mean_finetuned_notcausal_continuationMistral7B20ed0018b2a84fba09c4/checkpoints/checkpoint_005500'

# # Cross-Attention:
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/cross_att_5_last_layersMistral7Bdbbb7faebb2f32cf20e9/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/cross_att_fine_tuned_embedder_5_last_layersMistral7Bdbbb7faebb2f32cf20e9/checkpoints/checkpoint_007500'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/cross_att_finetuned_notcausal_continuationMistral7B20ed0018b2a84fba09c4/checkpoints/checkpoint_005000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/cross_att_pretrained_continuationMistral7B20ed0018b2a84fba09c4/checkpoints/checkpoint_008500'

with open(f'{ckpt_path}/params.json') as f:
    params = json.load(f)
print(params)

model_name = 'Mistral7B' # Mistral7B, Llama3.2-3B, Gemma7B
device = 'cuda' if torch.cuda.is_available() else 'cpu'
w_embeds = True
max_batch_size = 4

# variant = '7b' if model_name == 'Gemma7B' else None

### Modify old params

In [ ]:
with open(ckpt_path + '/params.json') as f:
    params = json.load(f)
print(params)
# if 'do_pool'  not in params.keys():
if 'n_truncated_layers' in params['pooling_module'].keys():
    params['n_truncated_layers'] = params['pooling_module']['n_truncated_layers']
    del params['pooling_module']['n_truncated_layers']
    

if params['cross_att'] is not None:
    print('here')
    params['normalize_embeddings'] = True if params['cross_att'] else False
    if params['start_cross_att'] is None:
        del params['start_cross_att']
    else:
        params['cross_att_layers'] = 32 - params["start_cross_att"]
        del params['start_cross_att']
    params['do_pool'] = False if params['cross_att'] else True
else:
    params['do_pool'] = True
print(params)
with open(ckpt_path + '/params.json', 'w') as f:
    json.dump(params, f)

        


### Reconstruction

In [ ]:
for param in tests:
    print('Param:', param)
    if param['w_embeds']:
        pipeline.pipeline_args.w_embeds = True
    else:
        pipeline.pipeline_args.w_embeds = False
    
    final_valid_prompts = [passage.split(' ')[0] for passage in valid_passage][2] 
    text_valid_conditioning = [passage[:100] for passage in valid_passage][2]
    print('Prompt', final_valid_prompts, ' | Passage', text_valid_conditioning)
    generated_sequence = pipeline.generate(prompts = final_valid_prompts, 
                                        text_conditioning = text_valid_conditioning, 
                                        temperature = param['temperature'], 
                                        max_tokens = max_tokens,
                                        truncate_double_space = False)
    print('Valid  word', generated_sequence)
    
    final_valid_prompts = ['' for passage in train_passage][1]
    text_valid_conditioning = [passage[:100] for passage in valid_passage][2]
    generated_sequence = pipeline.generate(prompts = final_valid_prompts, 
                                        text_conditioning = text_valid_conditioning, 
                                        temperature = param['temperature'], 
                                        max_tokens = max_tokens,
                                        truncate_double_space = False)
    print('Valid  empty', generated_sequence)

    final_train_prompts =  [passage.split(' ')[0] for passage in train_passage][1] 
    text_train_conditioning = [passage[:100] for passage in train_passage][1]
    print('Prompt', final_train_prompts, ' | Passage', text_train_conditioning)
    generated_sequence = pipeline.generate(prompts = final_train_prompts, 
                                       text_conditioning = text_train_conditioning, 
                                       temperature = param['temperature'], 
                                       max_tokens = max_tokens,
                                       truncate_double_space = False)
    print('Train word', generated_sequence)
    final_train_prompts = ['' for passage in train_passage][1] 
    text_train_conditioning = [passage[:100] for passage in train_passage][1]
    generated_sequence = pipeline.generate(prompts = final_train_prompts, 
                                       text_conditioning = text_train_conditioning, 
                                       temperature = param['temperature'], 
                                       max_tokens = max_tokens,
                                       truncate_double_space = False)
    print('Train empty', generated_sequence)

    

In [ ]:
# 1 information in the doc which enables to answer the question but not good response often in-context
# 2 information in the doc which enables to answer the question and good response often in-context
# 3 Hard negative passage
# 4 Same

# prompt_prefix = "Query: who wrote the song photograph by ringo starr\nAnswer: Ringo Starr\n\nQuery: who is playing the halftime show at super bowl 2016\nAnswer: Coldplay\n\nQuery: where was the world economic forum held this year\nAnswer: Davos\n\nQuery: where are the giant redwoods located in california\nAnswer: Humboldt County\n\nQuery: who has made the most premier league appearances\nAnswer: Gareth Barry\n\nQuery: "
# prompts = ['who has most followers on instagram in world','who did the united states win its independence from', 'locations for the film an englishman who went up a hill', 'who is the valley of the dolls based on']
# final_prompts = [prompt_prefix + prompt + '\nAnswer:' for prompt in prompts]

# text_conditioning = ["This list contains the top 50 accounts with the most followers on the photo and video-sharing social platform Instagram. As of July 2019, the most followed user is Instagram's own account, with over 308 million followers. Cristiano Ronaldo is the most followed individual, with over 177 million followers. Fifteen accounts have exceeded 100 million followers on the site.",
#                      "During the American Revolution, the legal separation of the thirteen colonies from Great Britain in 1776 actually occurred on July 2, when the Second Continental Congress voted to approve a resolution of independence that had been proposed in June by Richard Henry Lee of Virginia declaring the United States independent from Great Britain's rule. After voting for independence, Congress turned its attention to the Declaration of Independence, a statement explaining this decision, which had been prepared by a Committee of Five, with Thomas Jefferson as its principal author. Congress debated and revised the wording of the Declaration, finally approving it two days later on July 4. A day earlier, John Adams had written to his wife Abigail",
#                      'The village was a primary location for the making of the film \"The Englishman Who Went Up a Hill But Came Down a Mountain\", which starred Hugh Grant. The hilltop scenes were filmed on the Gyrn, the long hill that overlooks the village. It was also featured in \"Monk\'s Hood\", an episode of \"The Cadfael Chronicles\"',
#                      'Valley of the Dolls is the first novel by American writer Jacqueline Susann. Published in 1966, the book was the biggest selling novel of its year. To date, it has sold more than 31 million copies, making it one of the best-selling works in publishing history.']

# answers = ['Instagram','Great Britain',"Llansilin in Powys",["Judy Garland", "Carole Landis", "Dean Martin", "Ethel Merman"]]

n_passages = 4
eval_data = '/lustre/scwpod02/client/kyutai-interns/datasets/modular_finetuning/enwiki-20220120_valid.jsonl'
train_data = '/lustre/scwpod02/client/kyutai-interns/datasets/modular_finetuning/enwiki-20220120_train.jsonl'
train_passage = []
valid_passage = []
with open(train_data, 'r') as f:
    for i, line in enumerate(f):
        if i == n_passages:
            break
        train_passage.append(json.loads(line)['text'].split('\n\n')[1])
        
with open(eval_data, 'r') as f:
    for i, line in enumerate(f):
        if i == n_passages:
            break
        valid_passage.append(json.loads(line)['text'].split('\n\n')[1])
        
tests = [{'w_embeds': True, 'temperature': 0 },  {'w_embeds': True, 'temperature': 0.7 }, {'w_embeds': False, 'temperature': 0.7 }]
# print('Train passage:', train_passage)
# print('Valid passage:', valid_passage)

In [ ]:
conditioning = ['Kyutai is a non-profit laboratory dedicated to open research in AI, founded in November 2023 by the iliad Group, CMA CGM and Schmidt Sciences. Launched with an initial team of six leading scientists, who have all worked with Big Tech labs in the USA, Kyutai continues to recruit at the highest level, and also offers internships to research Master’s degree students.']*4
prompts = ['who are the founders of Kyutai?', 'when was Kyutai founded?', 'how many scientists were in the initial team?', 'what does Kyutai offer to research Master’s degree students?']
if w_embeds:
    pipeline.pipeline_args.w_embeds = True
else:
    pipeline.pipeline_args.w_embeds = False
generated_sequence = pipeline.generate(prompts = prompts,
                                      text_conditioning = conditioning,
                                      temperature = 0.5, 
                                      max_tokens =200,
                                      truncate_double_space = False)
# random_flip, put the number of the token to flip. 
print(generated_sequence)

if w_embeds:
    pipeline.pipeline_args.w_embeds = True
else:
    pipeline.pipeline_args.w_embeds = False
generated_sequence, logprobs = pipeline.generate(prompts = ['who has most followers on Instagram in world?'],
                                      text_conditioning = ["This list contains the top 50 accounts with the most followers on the photo and video-sharing social platform Instagram. As of July 2019, the most followed user is Instagram's own account, with over 308 million followers. Cristiano Ronaldo is the most followed individual, with over 177 million followers."],
                                      temperature = 0.4, 
                                      max_tokens =200,
                                      truncate_double_space = False)
print(generated_sequence)


### Continuation

In [ ]:
# Continuation
for param in tests:
    print('Param:', param)
    if param['w_embeds']:
        pipeline.pipeline_args.w_embeds = True
    else:
        pipeline.pipeline_args.w_embeds = False
    
    final_valid_prompts = [passage[100:].split(' ')[0] for passage in valid_passage][2] 
    text_valid_conditioning = [passage[:100] for passage in valid_passage][2]
    print('Passage', text_valid_conditioning, ' | Truth', [passage[100:200] for passage in valid_passage][2] )
    generated_sequence = pipeline.generate(prompts = final_valid_prompts, 
                                        text_conditioning = text_valid_conditioning, 
                                        temperature = param['temperature'], 
                                        max_tokens = max_tokens,
                                        truncate_double_space = False)
    print('Valid  word', generated_sequence)
    
    final_valid_prompts = ['' for passage in train_passage][2]
    text_valid_conditioning = [passage[:100] for passage in valid_passage][2]
    generated_sequence = pipeline.generate(prompts = final_valid_prompts, 
                                        text_conditioning = text_valid_conditioning, 
                                        temperature = param['temperature'], 
                                        max_tokens = max_tokens,
                                        truncate_double_space = False)
    print('Valid  empty', generated_sequence)

    final_train_prompts =  [passage[100:].split(' ')[0] for passage in train_passage][1] 
    text_train_conditioning = [passage[:100] for passage in train_passage][1]
    print('Passage', text_train_conditioning, ' | Truth', [passage[100:200] for passage in train_passage][1] )
    generated_sequence = pipeline.generate(prompts = final_train_prompts, 
                                       text_conditioning = text_train_conditioning, 
                                       temperature = param['temperature'], 
                                       max_tokens = max_tokens,
                                       truncate_double_space = False)
    print('Train word', generated_sequence)
    final_train_prompts = ['' for passage in train_passage][1] 
    text_train_conditioning = [passage[:100] for passage in train_passage][1]
    generated_sequence = pipeline.generate(prompts = final_train_prompts, 
                                       text_conditioning = text_train_conditioning, 
                                       temperature = param['temperature'], 
                                       max_tokens = max_tokens,
                                       truncate_double_space = False)
    print('Train empty', generated_sequence)


In [ ]:

# Tested
# run_name = '128_SL_FN_False_0_MLP_False_CA_False_DB'
# run_name = '128_SL_FN_False_0_MLP_True_CA_16_CAL_False_SKV_False_DB'
# run_name = '128_SL_FN_False_0_MLP_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_False_0_MLP_True_CA_24_CAL_False_SKV_True_DB'

# Finished runs:

# run_name = '128_SL_FN_Truemean_1_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_True_DB' # 008500
# run_name = '128_SL_FN_Truemean_3_MLP_4_TRUNC_True_CA_5_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truemean_3_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truemean_3_MLP_4_TRUNC_True_CA_24_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truemean_1_MLP_4_TRUNC_True_CA_5_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truemean_1_MLP_4_TRUNC_True_CA_24_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truemean_0_MLP_4_TRUNC_True_CA_5_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truemean_0_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truemean_0_MLP_4_TRUNC_True_CA_24_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truelatent_attention_0_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truelatent_attention_3_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Trueeos_0_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Trueeos_3_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truereversed_latent_atttention_0_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truereversed_latent_attention_3_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truemean_0_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_False_DB'
# run_name = '128_SL_FN_Truemean_3_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_False_DB'
# run_name = '128_SL_FN_Truemean_0_MLP_8_TRUNC_True_CA_16_CAL_False_SKV_False_DB'
# run_name = '128_SL_FN_Truemean_3_MLP_8_TRUNC_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_Truemean_3_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_False_DB'
# run_name = '128_SL_FN_Truemean_3_MLP_4_TRUNC_True_CA_16_CAL_False_SKV_True_DB_dist_process'
# run_name = '128_SL_FN_Truemean_3_MLP_4_TRUNC_False_CA_False_DB'
# run_name = '128_SL_FN_Truemean_3_MLP_8_TRUNC_False_CA_False_DB'
# run_name = '128_SL_FN_Truemean_0_MLP_4_TRUNC_False_CA_False_DB'
# run_name = '128_SL_FN_Truemean_0_MLP_8_TRUNC_False_CA_False_DB'
# run_name = '128_SL_FN_False_3_MLP_True_CA_5_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_False_3_MLP_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_False_3_MLP_True_CA_24_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_False_1_MLP_True_CA_5_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_False_1_MLP_True_CA_16_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_False_1_MLP_True_CA_24_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_False_0_MLP_True_CA_5_CAL_False_SKV_True_DB'
# run_name = '128_SL_FN_False_3_MLP_True_CA_16_CAL_False_SKV_False_DB'
# run_name = '128_SL_FN_False_3_MLP_False_CA_False_DB'
# run_name = '128_SL_FN_False_3_MLP_True_CA_16_CAL_True_SKV_True_DB'
# run_name = '128_SL_FN_False_3_MLP_True_CA_16_CAL_False_SKV_True_DB_dist_process'